In [1]:
import torch
import glob
import pickle
import sys
import torchvision.transforms as T
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import numpy as np

from PIL import Image
sys.path.append("../../backend")

from image_encoder import ImageEncoder

In [2]:
DATA_PATH = "D:/projects/fyndart/artworks/"
MODEL_PATH = "../deep_encoder.pt"

In [3]:
MEAN = (0.48145466, 0.4578275, 0.40821073)
STD = (0.26862954, 0.26130258, 0.27577711)

T1 = Resize(224, interpolation=Image.BICUBIC)
T2 = CenterCrop(224)
T3 = lambda x: x.convert("RGB")
T4 = ToTensor()
T5 = Normalize(MEAN, STD)

def to_tensor(images):
    return torch.cat([Compose([T1, T2, T3, T4, T5])(image).unsqueeze(0) for image in images])

In [4]:
references = []

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
encoder = ImageEncoder(MODEL_PATH, device)

collection1 = glob.glob(DATA_PATH + '*.jpeg')
collection2 = glob.glob(DATA_PATH + '*.jpg')
for file in (collection1 + collection2):
    references.append(file)

chunk_size = 100
chunks = [references[i:i + chunk_size] for i in range(0, len(references), chunk_size)]

references = list(map(lambda x: x.replace("\\", "/"), references))
references = list(map(lambda x: x[x.rfind("/") + 1:], references))

In [5]:
embeddings = []
for index, chunk in enumerate(chunks):
    images = list(map(lambda x: Image.open(x), chunk))
    print("process chunk " + str(index))

    with torch.no_grad():
        tensor = to_tensor(images).to(device)
        for embedding in encoder(tensor).cpu().detach().numpy():
            embeddings.append(embedding)

process chunk 0
process chunk 1
process chunk 2
process chunk 3
process chunk 4
process chunk 5
process chunk 6
process chunk 7
process chunk 8
process chunk 9
process chunk 10
process chunk 11
process chunk 12
process chunk 13
process chunk 14
process chunk 15
process chunk 16
process chunk 17
process chunk 18
process chunk 19
process chunk 20
process chunk 21


In [7]:
import pickle
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'references': references, 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    references = stored_data['references']
    embeddings = stored_data['embeddings']

In [11]:
tensor1 = to_tensor([Image.open(collection1[0])]).to(device)
tensor2 = to_tensor([Image.open(collection1[1])]).to(device)

with torch.no_grad():
    embedding1 = encoder(tensor1)
    embedding2 = encoder(tensor2)

print(embedding1.shape)
print(embedding2.shape)
    
from torch.nn import CosineSimilarity
cos = CosineSimilarity()
print(cos(embedding1, embedding2))

torch.Size([1, 256, 7, 7])
torch.Size([1, 256, 7, 7])
tensor([[[0.7691, 0.7978, 0.7055, 0.5842, 0.6243, 0.5780, 0.4434],
         [0.7251, 0.6972, 0.6040, 0.7631, 0.7437, 0.5954, 0.5275],
         [0.6077, 0.6612, 0.7934, 0.7069, 0.7144, 0.6128, 0.5885],
         [0.6952, 0.7574, 0.7730, 0.6081, 0.6449, 0.6190, 0.6928],
         [0.5181, 0.6390, 0.8613, 0.7970, 0.7591, 0.7614, 0.6051],
         [0.7512, 0.6886, 0.8427, 0.8029, 0.7736, 0.5426, 0.4037],
         [0.5948, 0.4834, 0.8211, 0.9257, 0.9315, 0.7774, 0.5763]]],
       device='cuda:0')


In [8]:
from torch import Tensor

cos_scores = cos_sim(image_embedding, embeddings)[0]
top_results = torch.topk(cos_scores, k=5)

preds = top_results.values.tolist()
index = top_results.indices.tolist()

for result in zip(preds, index):
    print(f"({round(result[0], 4)}) {references[result[1]]}")

torch.Size([1, 256, 7, 7])
torch.Size([2123, 256, 7, 7])
torch.Size([1, 256, 7, 7])
torch.Size([2123, 256, 7, 7])


RuntimeError: self must be a matrix